# 推論モデルでの関数呼び出しの管理
OpenAIは現在、[推論モデル](https://platform.openai.com/docs/guides/reasoning?api-mode=responses)を使用した関数呼び出しを提供しています。推論モデルは論理的な思考の連鎖に従うように訓練されており、複雑または多段階のタスクにより適しています。

> _o3やo4-miniなどの推論モデルは、推論を実行するために強化学習で訓練されたLLMです。推論モデルは回答する前に考え、ユーザーに応答する前に長い内部思考の連鎖を生成します。推論モデルは複雑な問題解決、コーディング、科学的推論、およびエージェント的ワークフローのための多段階計画において優れています。また、軽量なコーディングエージェントであるCodex CLIにとって最適なモデルでもあります。_

ほとんどの場合、APIを介してこれらのモデルを使用することは非常にシンプルで、馴染みのある「チャット」モデルを使用することと同等です。

ただし、特に関数呼び出しなどの機能を使用する際には、留意すべきいくつかのニュアンスがあります。

このノートブックのすべての例では、会話状態の管理に便利な抽象化を提供する新しい[Responses API](https://community.openai.com/t/introducing-the-responses-api/1140929)を使用しています。ただし、ここでの原則は古いchat completions APIを使用する場合にも関連します。

## 推論モデルへのAPI呼び出し

In [1]:
# pip install openai
# Import libraries 
import json
from openai import OpenAI
from uuid import uuid4
from typing import Callable

client = OpenAI()
MODEL_DEFAULTS = {
    "model": "o4-mini", # 200,000 token context window
    "reasoning": {"effort": "low", "summary": "auto"}, # Automatically summarise the reasoning process. Can also choose "detailed" or "none"
}

Responses APIを使用して推論モデルに簡単な呼び出しを行ってみましょう。
低い推論努力を指定し、便利な`output_text`属性でレスポンスを取得します。
フォローアップの質問をして、`previous_response_id`を使用することで、OpenAIに会話履歴を自動的に管理させることができます。

In [2]:
response = client.responses.create(
    input="Which of the last four Olympic host cities has the highest average temperature?",
    **MODEL_DEFAULTS
)
print(response.output_text)

response = client.responses.create(
    input="what about the lowest?",
    previous_response_id=response.id,
    **MODEL_DEFAULTS
)
print(response.output_text)

Among the last four Summer Olympic host cities—Tokyo (2020), Rio de Janeiro (2016), London (2012) and Beijing (2008)—Rio de Janeiro has by far the warmest climate. Average annual temperatures are roughly:

• Rio de Janeiro: ≈ 23 °C  
• Tokyo: ≈ 16 °C  
• Beijing: ≈ 13 °C  
• London: ≈ 11 °C  

So Rio de Janeiro has the highest average temperature.
Among those four, London has the lowest average annual temperature, at about 11 °C.


とても簡単です！

私たちは比較的複雑な質問をしており、モデルが計画を立てて段階的に進める必要があるかもしれませんが、この推論は私たちには隠されています - 単に少し長く待ってから応答が表示されるだけです。

しかし、出力を詳しく調べると、モデルがモデルのコンテキストウィンドウに含まれているが、エンドユーザーには公開されていない隠された「推論」トークンのセットを使用していることがわかります。
これらのトークンと推論の要約（ただし使用された実際のトークンではない）を応答で確認することができます。

In [3]:
print(next(rx for rx in response.output if rx.type == 'reasoning').summary[0].text)
response.usage.to_dict()

**Determining lowest temperatures**

The user is asking about the lowest average temperatures of the last four Olympic host cities: Tokyo, Rio, London, and Beijing. I see London has the lowest average temperature at around 11°C. If I double-check the annual averages: Rio is about 23°C, Tokyo is around 16°C, and Beijing is approximately 13°C. So, my final answer is London with an average of roughly 11°C. I could provide those approximate values clearly for the user.


{'input_tokens': 136,
 'input_tokens_details': {'cached_tokens': 0},
 'output_tokens': 89,
 'output_tokens_details': {'reasoning_tokens': 64},
 'total_tokens': 225}

これらの推論トークンについて知っておくことは重要です。なぜなら、従来のチャットモデルと比べて、利用可能なコンテキストウィンドウをより早く消費することを意味するからです。

## カスタム関数の呼び出し
カスタムツールの使用も必要とする複雑なリクエストをモデルに求めた場合、何が起こるでしょうか？
* オリンピック開催都市についてさらに質問があるが、各都市のIDを含む内部データベースもあると想像してみましょう。
* モデルは結果を返す前に、推論プロセスの途中で私たちのツールを呼び出す必要がある可能性があります。
* ランダムなUUIDを生成する関数を作成し、モデルにこれらのUUIDについて推論するよう求めてみましょう。

In [4]:

def get_city_uuid(city: str) -> str:
    """Just a fake tool to return a fake UUID"""
    uuid = str(uuid4())
    return f"{city} ID: {uuid}"

# The tool schema that we will pass to the model
tools = [
    {
        "type": "function",
        "name": "get_city_uuid",
        "description": "Retrieve the internal ID for a city from the internal database. Only invoke this function if the user needs to know the internal ID for a city.",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {"type": "string", "description": "The name of the city to get information about"}
            },
            "required": ["city"]
        }
    }
]

# This is a general practice - we need a mapping of the tool names we tell the model about, and the functions that implement them.
tool_mapping = {
    "get_city_uuid": get_city_uuid
}

# Let's add this to our defaults so we don't have to pass it every time
MODEL_DEFAULTS["tools"] = tools

response = client.responses.create(
    input="What's the internal ID for the lowest-temperature city?",
    previous_response_id=response.id,
    **MODEL_DEFAULTS)
print(response.output_text)


今回は`output_text`が取得できませんでした。レスポンスの出力を確認してみましょう。

In [5]:
response.output

[ResponseReasoningItem(id='rs_68246219e8288191af051173b1d53b3f0c4fbdb0d4a46f3c', summary=[], type='reasoning', status=None),
 ResponseFunctionToolCall(arguments='{"city":"London"}', call_id='call_Mx6pyTjCkSkmASETsVASogoC', name='get_city_uuid', type='function_call', id='fc_6824621b8f6c8191a8095df7230b611e0c4fbdb0d4a46f3c', status='completed')]

推論ステップと併せて、モデルは正常にツール呼び出しの必要性を特定し、関数呼び出しに送信する指示を返しました。

関数を呼び出して結果をモデルに送信し、推論を継続できるようにしましょう。
関数のレスポンスは特別な種類のメッセージなので、次のメッセージを特別な種類の入力として構造化する必要があります：

```json
{
    "type": "function_call_output",
    "call_id": function_call.call_id,
    "output": tool_output
}
```

In [6]:
# Extract the function call(s) from the response
new_conversation_items = []
function_calls = [rx for rx in response.output if rx.type == 'function_call']
for function_call in function_calls:
    target_tool = tool_mapping.get(function_call.name)
    if not target_tool:
        raise ValueError(f"No tool found for function call: {function_call.name}")
    arguments = json.loads(function_call.arguments) # Load the arguments as a dictionary
    tool_output = target_tool(**arguments) # Invoke the tool with the arguments
    new_conversation_items.append({
        "type": "function_call_output",
        "call_id": function_call.call_id, # We map the response back to the original function call
        "output": tool_output
    })

In [7]:
response = client.responses.create(
    input=new_conversation_items,
    previous_response_id=response.id,
    **MODEL_DEFAULTS
)
print(response.output_text)


The internal ID for London is 816bed76-b956-46c4-94ec-51d30b022725.


これはここでは素晴らしく動作します - モデルが応答するために必要なのは単一の関数呼び出しだけであることがわかっているためです - しかし、推論を完了するために複数のツール呼び出しを実行する必要がある状況も考慮する必要があります。

2番目の呼び出しを追加して、ウェブ検索を実行してみましょう。

OpenAIのウェブ検索ツールは、推論モデルでは標準では利用できません（2025年5月時点 - これは近いうちに変更される可能性があります）が、4o miniまたは他のウェブ検索対応モデルを使用してカスタムウェブ検索関数を作成するのはそれほど難しくありません。

In [8]:
def web_search(query: str) -> str:
    """Search the web for information and return back a summary of the results"""
    result = client.responses.create(
        model="gpt-4o-mini",
        input=f"Search the web for '{query}' and reply with only the result.",
        tools=[{"type": "web_search_preview"}],
    )
    return result.output_text

tools.append({
        "type": "function",
        "name": "web_search",
        "description": "Search the web for information and return back a summary of the results",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {"type": "string", "description": "The query to search the web for."}
            },
            "required": ["query"]
        }
    })
tool_mapping["web_search"] = web_search


## 複数の関数を順次実行する

一部のOpenAIモデルは`parallel_tool_calls`パラメータをサポートしており、これによりモデルが関数の配列を返し、それらを並列で実行することができます。しかし、推論モデルは順次実行する必要がある一連の関数呼び出しを生成する場合があります。特に、一部のステップが前のステップの結果に依存する場合があるためです。

そのため、任意の複雑な推論ワークフローを処理するために使用できる一般的なパターンを定義する必要があります：

* 会話の各ステップで、ループを初期化する
* レスポンスに関数呼び出しが含まれている場合、推論が進行中であると仮定し、関数の結果（および中間的な推論）をモデルにフィードバックして、さらなる推論を行う必要がある
* 関数呼び出しがなく、代わりにタイプが'message'の`Response.output`を受信した場合、エージェントが推論を完了したと安全に仮定でき、ループから抜け出すことができる

In [9]:
# Let's wrap our logic above into a function which we can use to invoke tool calls.
def invoke_functions_from_response(response,
                                   tool_mapping: dict[str, Callable] = tool_mapping
                                   ) -> list[dict]:
    """Extract all function calls from the response, look up the corresponding tool function(s) and execute them.
    (This would be a good place to handle asynchroneous tool calls, or ones that take a while to execute.)
    This returns a list of messages to be added to the conversation history.
    """
    intermediate_messages = []
    for response_item in response.output:
        if response_item.type == 'function_call':
            target_tool = tool_mapping.get(response_item.name)
            if target_tool:
                try:
                    arguments = json.loads(response_item.arguments)
                    print(f"Invoking tool: {response_item.name}({arguments})")
                    tool_output = target_tool(**arguments)
                except Exception as e:
                    msg = f"Error executing function call: {response_item.name}: {e}"
                    tool_output = msg
                    print(msg)
            else:
                msg = f"ERROR - No tool registered for function call: {response_item.name}"
                tool_output = msg
                print(msg)
            intermediate_messages.append({
                "type": "function_call_output",
                "call_id": response_item.call_id,
                "output": tool_output
            })
        elif response_item.type == 'reasoning':
            print(f'Reasoning step: {response_item.summary}')
    return intermediate_messages

それでは、先ほど説明したループの概念を実演してみましょう。

In [10]:
initial_question = (
    "What are the internal IDs for the cities that have hosted the Olympics in the last 20 years, "
    "and which of those cities have recent news stories (in 2025) about the Olympics? "
    "Use your internal tools to look up the IDs and the web search tool to find the news stories."
)

# We fetch a response and then kick off a loop to handle the response
response = client.responses.create(
    input=initial_question,
    **MODEL_DEFAULTS,
)
while True:   
    function_responses = invoke_functions_from_response(response)
    if len(function_responses) == 0: # We're done reasoning
        print(response.output_text)
        break
    else:
        print("More reasoning required, continuing...")
        response = client.responses.create(
            input=function_responses,
            previous_response_id=response.id,
            **MODEL_DEFAULTS
        )

Reasoning step: []
Invoking tool: get_city_uuid({'city': 'Beijing'})
More reasoning required, continuing...
Reasoning step: []
Invoking tool: get_city_uuid({'city': 'London'})
More reasoning required, continuing...
Reasoning step: []
Invoking tool: get_city_uuid({'city': 'Rio de Janeiro'})
More reasoning required, continuing...
Reasoning step: []
Invoking tool: get_city_uuid({'city': 'Tokyo'})
More reasoning required, continuing...
Reasoning step: []
Invoking tool: get_city_uuid({'city': 'Paris'})
More reasoning required, continuing...
Reasoning step: []
Invoking tool: get_city_uuid({'city': 'Turin'})
More reasoning required, continuing...
Reasoning step: []
Invoking tool: get_city_uuid({'city': 'Vancouver'})
More reasoning required, continuing...
Reasoning step: []
Invoking tool: get_city_uuid({'city': 'Sochi'})
More reasoning required, continuing...
Reasoning step: []
Invoking tool: get_city_uuid({'city': 'Pyeongchang'})
More reasoning required, continuing...
Reasoning step: []
Invok

## 手動会話オーケストレーション
ここまで順調です！モデルが実行を一時停止して関数を実行してから続行する様子を見るのは本当にクールです。
実際には、上記の例はかなり単純で、本番環境のユースケースはもっと複雑になる可能性があります：
* コンテキストウィンドウが大きくなりすぎて、古くて関連性の低いメッセージを削除したり、これまでの会話を要約したりしたい場合があります
* ユーザーが会話を前後に移動して回答を再生成できるようにしたい場合があります
* 監査目的でOpenAIのストレージとオーケストレーションに依存するのではなく、独自のデータベースにメッセージを保存したい場合があります
* など

このような状況では、会話を完全にコントロールしたい場合があります。`previous_message_id`を使用する代わりに、APIを「ステートレス」として扱い、毎回モデルに入力として送信する会話アイテムの配列を作成・維持することができます。

これにはReasoningモデル特有の考慮すべき微妙な点があります。
* 特に、会話履歴において推論と関数呼び出しの応答を保持することが不可欠です。
* これにより、モデルはどのような思考の連鎖ステップを実行したかを追跡できます。これらが含まれていない場合、APIはエラーを返します。

上記の例を再度実行し、メッセージを自分たちでオーケストレーションし、トークン使用量を追跡してみましょう。

---
*以下のコードは読みやすさのために構造化されています - 実際には、エッジケースを処理するためのより洗練されたワークフローを検討することをお勧めします*

In [11]:
# Let's initialise our conversation with the first user message
total_tokens_used = 0
user_messages = [
    (
        "Of those cities that have hosted the summer Olympic games in the last 20 years - "
        "do any of them have IDs beginning with a number and a temperate climate? "
        "Use your available tools to look up the IDs for each city and make sure to search the web to find out about the climate."
    ),
    "Great thanks! We've just updated the IDs - could you please check again?"
    ]

conversation = []
for message in user_messages:
    conversation_item = {
        "role": "user",
        "type": "message",
        "content": message
    }
    print(f"{'*' * 79}\nUser message: {message}\n{'*' * 79}")
    conversation.append(conversation_item)
    while True: # Response loop
        response = client.responses.create(
            input=conversation,
            **MODEL_DEFAULTS
        )
        total_tokens_used += response.usage.total_tokens
        reasoning = [rx.to_dict() for rx in response.output if rx.type == 'reasoning']
        function_calls = [rx.to_dict() for rx in response.output if rx.type == 'function_call']
        messages = [rx.to_dict() for rx in response.output if rx.type == 'message']
        if len(reasoning) > 0:
            print("More reasoning required, continuing...")
            # Ensure we capture any reasoning steps
            conversation.extend(reasoning)
            print('\n'.join(s['text'] for r in reasoning for s in r['summary']))
        if len(function_calls) > 0:
            function_outputs = invoke_functions_from_response(response)
            # Preserve order of function calls and outputs in case of multiple function calls (currently not supported by reasoning models, but worth considering)
            interleaved = [val for pair in zip(function_calls, function_outputs) for val in pair]
            conversation.extend(interleaved)
        if len(messages) > 0:
            print(response.output_text)
            conversation.extend(messages)
        if len(function_calls) == 0:  # No more functions = We're done reasoning and we're ready for the next user message
            break
print(f"Total tokens used: {total_tokens_used} ({total_tokens_used / 200_000:.2%} of o4-mini's context window)")

*******************************************************************************
User message: Of those cities that have hosted the summer Olympic games in the last 20 years - do any of them have IDs beginning with a number and a temperate climate? Use your available tools to look up the IDs for each city and make sure to search the web to find out about the climate.
*******************************************************************************
More reasoning required, continuing...
**Clarifying Olympic Cities**

The user is asking about cities that hosted the Summer Olympics in the last 20 years. The relevant years to consider are 2004 Athens, 2008 Beijing, 2012 London, 2016 Rio de Janeiro, and 2020 Tokyo. If we're considering 2025, then 2004 would actually be 21 years ago, so I should focus instead on the years from 2005 onwards. Therefore, the cities to include are Beijing, London, Rio, and Tokyo. I’ll exclude Paris since it hasn’t hosted yet.
Reasoning step: [Summary(text="**Clarif

## 概要
このクックブックでは、OpenAIの推論モデルと関数呼び出しを組み合わせて、ウェブ検索を含む外部データソースに依存する複数ステップのタスクを実行する方法を特定しました。

重要なことに、関数呼び出しプロセスにおける推論モデル特有のニュアンスを取り上げました。具体的には：
* モデルは複数の関数呼び出しや推論ステップを連続して実行することを選択する場合があり、一部のステップは前のステップの結果に依存する可能性がある
* これらのステップがいくつになるかを事前に知ることはできないため、ループを使用してレスポンスを処理する必要がある
* Responses APIは`previous_response_id`パラメータを使用してオーケストレーションを簡単にしますが、手動制御が必要な場合は、「思考の連鎖」を保持するために会話アイテムの正しい順序を維持することが重要

---

ここで使用した例はかなりシンプルですが、この技術をより実世界のユースケースに拡張できることを想像できるでしょう。例えば：

* 顧客の取引履歴と最近の対応記録を調べて、プロモーションオファーの対象かどうかを判断する
* 最近の取引ログ、位置情報データ、デバイスメタデータを呼び出して、取引が不正である可能性を評価する
* 内部HRデータベースを確認して、従業員の福利厚生利用状況、勤続年数、最近のポリシー変更を取得し、個人向けのHR質問に回答する
* 内部ダッシュボード、競合他社のニュースフィード、市場分析を読み取って、経営陣の重点分野に合わせた日次エグゼクティブブリーフィングを作成する